# Visualizations Notebook

The following notebook has as a purpose to show visualizations regarding the Education status at South America.

### Set up of libraries

In [1]:
import pandas as pd
import altair as alt
import requests

In [2]:
from utils import filters_countries_cols, translate_english_degrees

In [3]:
# Loading the data
raw = pd.read_csv("../data/raw/matricula.csv", encoding="latin1", sep=None, engine="python")


In [4]:
raw

,AÑO,TOTAL MATRÍCULA,TOTAL MATRÍCULA MUJERES,TOTAL MATRÍCULA HOMBRES,TOTAL MATRÍCULA NO BINARIOS O INDEFINIDOS,TOTAL MATRÍCULA PRIMER AÑO,TOTAL MATRÍCULA MUJERES PRIMER AÑO,TOTAL MATRÍCULA HOMBRES PRIMER AÑO,TOTAL MATRÍCULA NO BINARIOS O INDEFINIDOS PRIMER AÑO,CLASIFICACIÓN INSTITUCIÓN NIVEL 1,...,TES PARTICULAR SUBVENCIONADO,TES PARTICULAR PAGADO,TES CORP. DE ADMINISTRACIÓN DELEGADA,TES SERVICIO LOCAL EDUCACION,TOTAL TES,% COBERTURA TES,TIPO ESTABLECIMIENTO HC,TIPO ESTABLECIMIENTO TP,CLAS_EST ADULTO,CLAS_EST JOVEN
0,MAT_2025,245,179.0,66.0,NaN,94.0,67.0,27.0,NaN,Institutos Profesionales,...,162.0,2.0,11.0,7.0,228,"93,10%",142.0,86.0,43.0,185.0
1,MAT_2025,86,18.0,68.0,NaN,29.0,3.0,26.0,NaN,Institutos Profesionales,...,58.0,2.0,NaN,NaN,74,"86,00%",44.0,30.0,12.0,62.0
2,MAT_2025,15,3.0,12.0,NaN,NaN,NaN,NaN,NaN,Institutos Profesionales,...,7.0,NaN,1.0,NaN,12,"80,00%",8.0,4.0,5.0,7.0
3,MAT_2025,18,14.0,4.0,NaN,6.0,5.0,1.0,NaN,Universidades,...,10.0,2.0,NaN,NaN,15,"83,30%",14.0,1.0,NaN,15.0
4,MAT_2025,114,19.0,95.0,NaN,28.0,4.0,24.0,NaN,Institutos Profesionales,...,51.0,1.0,1.0,1.0,103,"90,40%",49.0,54.0,23.0,80.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264253,MAT_2007,18,13.0,5.0,NaN,18.0,13.0,5.0,NaN,Universidades,...,5.0,2.0,NaN,NaN,10,"55,60%",9.0,1.0,NaN,10.0
264254,MAT_2007,13,7.0,6.0,NaN,13.0,7.0,6.0,NaN,Universidades,...,4.0,1.0,NaN,NaN,7,"53,80%",5.0,2.0,NaN,7.0
264255,MAT_2007,16,10.0,6.0,NaN,NaN,NaN,NaN,NaN,Universidades,...,5.0,NaN,NaN,NaN,8,"50,00%",6.0,2.0,1.0,7.0
264256,MAT_2007,123,61.0,62.0,NaN,44.0,23.0,21.0,NaN,Universidades,...,33.0,8.0,NaN,NaN,58,"47,20%",50.0,8.0,4.0,54.0


In [5]:
# year column to number
raw["AÑO"] = raw["AÑO"].astype(str).str[-4:]
raw["AÑO"] = raw["AÑO"].astype(int)

# columns in lowercase
cols = [col.lower() for col in raw.columns]
raw.columns = cols

#

In [6]:
raw["carrera clasificación nivel 2"].unique()

array(['Carreras Técnicas', 'Doctorado', 'Carreras Profesionales',
       'Postítulo', 'Magister'], dtype=object)

# **First Chart: Evolution total students of undergrads, and grad students 2007-2025**

This line chart shows the total number of enrolled students for undergrads, grads and diploma students
in Chile for the period 2007-2025

In [7]:
# Getting the data
cols_1 = ["año", "nivel global"]
df_1 = raw.groupby(cols_1)["total matrícula"].sum().reset_index() 
df_1["nivel global"] = df_1.apply(translate_english_degrees, axis=1)
df_1.rename(columns = {"nivel global" : "Degree Type"}, inplace=True) 

# Chart number of enrolled during type
chart_1 = alt.Chart(df_1).mark_area().encode(
    x=alt.Y("año:O", axis= alt.Axis(title="Year"), scale=alt.Scale(2007,2025)),
    y=alt.Y("total matrícula:Q", axis= alt.Axis(title="Total enrolled")),
    color = "Degree Type:N").properties(
    title = "Total enrolled students by Degree Type for 2007-2025"
)

chart_1

alt.Chart(...)

# **Second Chart: Out of undergrads, relation of professional and technical undergrads**

The following chart shows the relation out of undergrads of students studying
technical careers and professional careers


In [8]:
# Degrees to show
def translates_professional_undegrads(row):
    value = row["Degree type of undergrads"]
    if value == "Carreras Profesionales":
        return "Professionals"
    elif value == "Carreras Técnicas":
        return "Technical"
    elif value == "Doctorados":
        return "PhD"
    elif value == "Postítulo":
        return "Diplomas"
    else:
        return "Masters"

degrees = ["Carreras Profesionales", "Carreras Técnicas"]

# Data
cols_2 = ["año", "carrera clasificación nivel 2"]
df_2 = raw.groupby(cols_2)["total matrícula"].sum().reset_index()

# Cleaning the data
df_2.rename(columns={"carrera clasificación nivel 2": "Degree type of undergrads"}, inplace=True)
df_2 = df_2[df_2["Degree type of undergrads"].isin(degrees)]
df_2["Degree type of undergrads"] = df_2.apply(translates_professional_undegrads, axis=1)

chart_2 = alt.Chart(df_2).mark_bar().encode(
    x = alt.X("año:O", axis=alt.Axis(title="Year")),
    y = alt.Y("sum(total matrícula)", axis=alt.Axis(title="Total enrolled")).stack("normalize"),
    color = "Degree type of undergrads").properties(
        title="Share of Technical and Professioanl enrolled undergrads"
    )

chart_2


alt.Chart(...)

# **Third Chart: Evolution Total male and female students of overall enrolled**

The following chart shows the growth of male and females students during 2007-2025


In [9]:
cols_3 = ["año"]
df_3 = raw.groupby(cols_3)[["total matrícula hombres", "total matrícula mujeres"]].sum().reset_index()



df_3 = df_3.melt(id_vars="año",
                 value_vars=["total matrícula hombres", "total matrícula mujeres"],
                 var_name="enrolled type",
                 value_name="total matrícula")

chart_3 = alt.Chart(df_3).mark_line().encode(
    x = alt.X("año:O", axis=alt.Axis(title="Year")),
    y = alt.Y("sum(total matrícula)", axis=alt.Axis(title="Total enrolled")),
    color = "enrolled type")

chart_3


alt.Chart(...)

# **Fourth Chart: Differences of study areas between men and women**

The following chart shows the ranking of top enrolled number of students of
knowkledge areas, comparing men and women for the 2007-2025 period

In [14]:
h = raw[raw["año"] == 2025]
h = h[["total matrícula mujeres", "área del conocimiento"]]
h = h.groupby(["área del conocimiento"])["total matrícula mujeres"].sum()
h



área del conocimiento
Administración y Comercio    149038.0
Agropecuaria                  28963.0
Arte y Arquitectura           48100.0
Ciencias Básicas              13373.0
Ciencias Sociales            112012.0
Derecho                       35281.0
Educación                    102659.0
Humanidades                    7005.0
Salud                        200937.0
Tecnología                    78255.0
Name: total matrícula mujeres, dtype: float64

In [16]:
y = raw[raw["año"] == 2025]
y = y[["total matrícula hombres", "área del conocimiento"]]
y = y.groupby(["área del conocimiento"])["total matrícula hombres"].sum()
y

área del conocimiento
Administración y Comercio    125319.0
Agropecuaria                  17047.0
Arte y Arquitectura           28279.0
Ciencias Básicas              18744.0
Ciencias Sociales             48170.0
Derecho                       26377.0
Educación                     34083.0
Humanidades                    4460.0
Salud                         72527.0
Tecnología                   304983.0
Name: total matrícula hombres, dtype: float64